**ALLER AU GPU EN PREMIER ET METTRE T4!!!**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import json

file_path = "/content/drive/MyDrive/Colab Notebooks/Memoire_M1/Test-format/Corona2.json"
with open(file_path, "r", encoding="utf-8") as file:
    data = json.load(file)

# Affichage des 2 premières entrées de la liste "examples" (si elle existe) afin de vérifier la bonne lecture des données
if "examples" in data and isinstance(data["examples"], list):
    print(json.dumps(data["examples"][:1], indent=4, ensure_ascii=False))
else:
    print("Format de données inattendu :", type(data))

In [ ]:
import os

# Définir le chemin du dossier de sortie
output_dir = "/content/drive/MyDrive/Colab Notebooks/Memoire_M1/Test-format/Sortie_3"

# Créer le dossier s'il n'existe pas
os.makedirs(output_dir, exist_ok=True)

print(f"Dossier de sortie créé à : {output_dir}")

Début du tuto

In [ ]:
#Installation spacy-transformers
!pip install spacy[transformers]

In [ ]:
import json
with open(file_path, "r", encoding="utf-8") as file:
    data = json.load(file)
print(data['examples'][0])

On a vérifié le format des données. Dans la cellule d'après on crée training_data qui contiendra uniquement les classes et les annotations, autrement dit le texte (content) et les annotations (start,end, label).

In [ ]:
training_data = {'classes' : ['MEDICINE', "MEDICALCONDITION", "PATHOGEN"], 'annotations' : []}
for example in data['examples']:
  temp_dict = {}
  temp_dict['text'] = example['content']
  temp_dict['entities'] = []
  for annotation in example['annotations']:
    start = annotation['start']
    end = annotation['end']
    label = annotation['tag_name'].upper()
    temp_dict['entities'].append((start, end, label))
  training_data['annotations'].append(temp_dict)

print(training_data['annotations'][0])

# Chemin du fichier training
output_path = os.path.join(output_dir, "training_data.json")

# Sauvegarde en JSON
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(training_data, f, ensure_ascii=False, indent=4)

print(f"Fichier enregistré : {output_path}")

Remarques sur le format produit


*   dictionnaire à 2 clés principales "text" et "entities"
*   chaque entité suit le format (start, end, label)
* les labels sont tous en majuscules (pré-requis de spacy ??)
*"text" contient un paragraphe annoté
*"entities" contient plusieurs annotations correspondant aux termes médicaux extraits
Maintenant, il faut convertir ces données ci en format binaire DocBin

In [ ]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm

nlp = spacy.blank("en") # load a new spacy model
doc_bin = DocBin() # create a DocB

In [ ]:
# #Code totalement issue du tuto, on voit qu'il y a des entités skippées

# from spacy.util import filter_spans

# for training_example  in tqdm(training_data['annotations']):
#     text = training_example['text']
#     labels = training_example['entities']
#     doc = nlp.make_doc(text)
#     ents = []
#     for start, end, label in labels:
#         span = doc.char_span(start, end, label=label, alignment_mode="contract")
#         if span is None:
#             print("Skipping entity")
#         else:
#             ents.append(span)
#     filtered_ents = filter_spans(ents)
#     doc.ents = filtered_ents
#     doc_bin.add(doc)

# doc_bin.to_disk("training_data.spacy") # save the docbin object

# # Enregistrer le fichier dans le bon dossier
# output_path = os.path.join(output_dir, "training_data.spacy")
# doc_bin.to_disk(output_path)  # Sauvegarde dans le dossier de sortie

# print(f"Fichier enregistré avec succès : {output_path}")

Pourquoi certaines entités n'ont pas été traitées:


*   l'index start et end ne correspond pas exactement aux limites d'un mot (problème d'aligmement avec certains caractères)
*   mauvais traitement des caractères spéciaux et espaces vides qui modifient les index ?

In [ ]:
from spacy.util import filter_spans

# Vérifier les entités qui posent problème
for training_example in tqdm(training_data['annotations']):
    text = training_example['text']
    labels = training_example['entities']
    doc = nlp.make_doc(text)
    ents = []

    for start, end, label in labels:
        span = doc.char_span(start, end, label=label, alignment_mode="expand")  # Changer "contract" en "expand" afin de capturer l'entité e nentier au lieu de couper en plein milieu
        if span is None: # Si le problème n'est pas réslu et que certaines entités ne sont toujours pas traitées
            print(f"Skipping entity: '{text[start:end]}' ({label}) | Indices: {start}-{end}")
        else:
            ents.append(span)

    filtered_ents = filter_spans(ents)
    doc.ents = filtered_ents
    doc_bin.add(doc)

doc_bin.to_disk("training_data.spacy")
print("Enregistrement terminé")

# Enregistrer le fichier
output_path = os.path.join(output_dir, "training_data.spacy")
doc_bin.to_disk(output_path)  # Sauvegarde dans dossier sortie

print(f"Fichier enregistré avec succès : {output_path}")

Ensuite on génère le fichier de configuration pour spacy:


1.   en anglais (en)
2.   composants (ner uniquement ici pas comme dans le tuto car problème lors de l'entraînement avec "tagger, parser" en plus)
3.   optimisation accuracy
4.   utilisation GPU transformers

Utilisation de la commande pour ne pas avoir à se soucier d'un nouveau chemin de fichier vers config.cfg.
Première étape = générer la configuration souhaitée
Deuxième étape = compléter le fichier avec les valeurs par défaut (ne semblait pas nécessaire vu le message affiché).

In [ ]:
!python -m spacy init config base_config.cfg --lang en --pipeline ner --optimize accuracy --gpu

**Génération du fichier base config en fr pour nos données**

In [ ]:
!python -m spacy init fill-config base_config.cfg config.cfg

Ensuite pour commencer l'entraînement je dois vérifier que GPU est disponible: je suis ici sur colab donc on doit faire : exécution > modifier le type d'exécution et choisi dans "accélérateur matériel" la case GPU (à noter que le nombre d'utilisation est limité donc mieux sur ordi mais j'ai un problème avec le mien) + TRES IMPORTANT, IL FAUT FAIRE RETOURNER LES AUTRES CELLULES DONC A FAIRE AU DEBUT DU PROGRAMME

In [ ]:
import torch
print("GPU disponible :", torch.cuda.is_available())

import spacy
print("spaCy GPU support :", spacy.prefer_gpu())

!nvidia-smi

On lance donc l'entraînement avec l'option GPU !
(problème pour enregistrer à distance dans drive donc on doit tout télécharger c'est un peu long à faire ....)

In [ ]:
!python -m spacy train config.cfg --output ./ --paths.train training_data.spacy --paths.dev training_data.spacy --gpu-id 0

32 minutes sur google colab

In [ ]:
nlp_ner = spacy.load("model-best")

doc = nlp_ner("Antiretroviral therapy (ART) is recommended for all HIV-infected\
individuals to reduce the risk of disease progression.\nART also is recommended \
for HIV-infected individuals for the prevention of transmission of HIV.\nPatients \
starting ART should be willing and able to commit to treatment and understand the\
benefits and risks of therapy and the importance of adherence. Patients may choose\
to postpone therapy, and providers, on a case-by-case basis, may elect to defer\
therapy on the basis of clinical and/or psychosocial factors.")

colors = {"PATHOGEN": "#F67DE3", "MEDICINE": "#7DF6D9", "MEDICALCONDITION":"#FFFFFF"}
options = {"colors": colors}

spacy.displacy.render(doc, style="ent", options= options, jupyter=True)